### Traitement des fichiers de prix Future

In [11]:
# Librairies nécessaires

import pandas as pd
from datetime import datetime
import pytz

In [21]:
# Chemin d'accès au fichier CSV
file_path = r"C:\Users\Meryam GHASSIR\UPLE Dropbox\Achat & Appro\SAUVEGARDE OUTILS\Calcul-Portefeuille\P50\Injection\test-models\Consommation\EEX"
output = r"C:\Users\Meryam GHASSIR\UPLE Dropbox\Achat & Appro\SAUVEGARDE OUTILS\Calcul-Portefeuille\P50\Injection\test-models\Consommation\Output"

In [24]:
def lec_eex(name_file):
    """
    Fonction pour lire un fichier CSV et retourner un DataFrame.
    
    :param name_file: Nom du fichier CSV à lire
    :return: DataFrame contenant les données du fichier
    """
    global file_path
    df = pd.read_csv(file_path + "\\" + name_file, sep=';', header=3)
    df = df.iloc[4:]  # Suppression des lignes inutiles
    return df

future_22 = lec_eex("PowerFutureResults_FR_20221228.csv")
future_23 = lec_eex("PowerFutureResults_FR_20231227.csv")
future_24 = lec_eex("PowerFutureResults_FR_20241227.csv")

print(future_24.head())

  # Data type(PR) Product                            Long Name Maturity  \
4              PR    F724  EEX French Power Base Day 24 Future  2024-12   
5              PR    F725  EEX French Power Base Day 25 Future  2024-12   
6              PR    F726  EEX French Power Base Day 26 Future  2024-12   
7              PR    F727  EEX French Power Base Day 27 Future  2024-12   
8              PR    F728  EEX French Power Base Day 28 Future  2024-12   

  Delivery Start Delivery End Open Price  Timestamp Open Price High Price  \
4     2024-12-24   2024-12-24        NaN                   NaN        NaN   
5     2024-12-25   2024-12-25        NaN                   NaN        NaN   
6     2024-12-26   2024-12-26        NaN                   NaN        NaN   
7     2024-12-27   2024-12-27        NaN                   NaN        NaN   
8     2024-12-28   2024-12-28      119,5  2024-12-27T09:55:44Z      119,5   

   Timestamp High Price  ...  Timestamp Last Price Settlement Price  \
4              

In [25]:
# Retraitement des données
def traitement_eex(df):
    """
    Suppression des colonnes inutiles et 
    conversion des types de données en fonctions des colonnes.
    """
    # Suppression des colonnes inutiles
    df = df.drop(columns=['# Data type(PR)', 'Product', 'Unit of Prices', 'Lot Size', 'Traded Lots', 'Number of Trades', 'Traded Volume', 'Open Interest Lots', 'Open Interest Volume', 'Unit of Volumes'])

    # Conversion des types de données
    df['Maturity'] = pd.to_datetime(df['Maturity'], format='%Y-%m')
    df['Delivery Start'] = pd.to_datetime(df['Delivery Start'], infer_datetime_format=True)
    df['Delivery End'] = pd.to_datetime(df['Delivery End'], infer_datetime_format=True)
    
    # Conversion des colonnes numériques - remplacement de la virgule par un point
    numeric_columns = ['Open Price', 'High Price', 'Low Price', 'Last Price', 'Settlement Price']
    for col in numeric_columns:
        df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df

future_22 = traitement_eex(future_22)
future_23 = traitement_eex(future_23)
future_24 = traitement_eex(future_24)


C:\Users\Meryam GHASSIR\AppData\Local\Temp\ipykernel_4724\3688822996.py:12: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['Delivery Start'] = pd.to_datetime(df['Delivery Start'], infer_datetime_format=True)
C:\Users\Meryam GHASSIR\AppData\Local\Temp\ipykernel_4724\3688822996.py:12: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Delivery Start'] = pd.to_datetime(df['Delivery Start'], infer_datetime_format=True)
C:\Users\Meryam GHASSIR\AppData\Local\Temp\ipykernel_4724\3688822996.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.p

In [26]:
# Format spécifique de date pour 3 colonnes - Retraitement à part
def convertir_col_iso(df, colonnes):
    # Convertir chaque colonne en datetime UTC, puis en heure locale française
    for col in colonnes:
        # Utiliser format='ISO8601' pour les timestamps au format 2022-12-28T09:35:42Z
        # et errors='coerce' pour convertir les valeurs invalides en NaT
        df[col] = pd.to_datetime(df[col], format='ISO8601', errors='coerce', utc=True)
        df[col] = df[col].dt.tz_convert('Europe/Paris')
        df[col] = df[col].dt.strftime('%d/%m/%Y %H:%M:%S')
    return df

colonnes = ['Timestamp Open Price', 'Timestamp High Price', 'Timestamp Low Price', 'Timestamp Last Price']

future_22 = convertir_col_iso(future_22, colonnes)
future_23 = convertir_col_iso(future_23, colonnes) 
future_24 = convertir_col_iso(future_24, colonnes)

In [27]:
def tocsv(df, name_file):
    """
    Fonction pour sauvegarder un DataFrame dans un fichier CSV.
    
    :param df: DataFrame à sauvegarder
    :param name_file: Nom du fichier CSV de sortie
    """
    df.to_csv(output + "\\" + name_file, sep=';', index=False)
    return df

# Sauvegarde des DataFrames dans des fichiers CSV
tocsv(future_22, "future_22.csv")   
tocsv(future_23, "future_23.csv")
tocsv(future_24, "future_24.csv")

,Long Name,Maturity,Delivery Start,Delivery End,Open Price,Timestamp Open Price,High Price,Timestamp High Price,Low Price,Timestamp Low Price,Last Price,Timestamp Last Price,Settlement Price
4,EEX French Power Base Day 24 Future,2024-12-01,2024-12-24,2024-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.49
5,EEX French Power Base Day 25 Future,2024-12-01,2024-12-25,2024-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.79
6,EEX French Power Base Day 26 Future,2024-12-01,2024-12-26,2024-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.64
7,EEX French Power Base Day 27 Future,2024-12-01,2024-12-27,2024-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.54
8,EEX French Power Base Day 28 Future,2024-12-01,2024-12-28,2024-12-28,119.5,27/12/2024 10:55:44,119.5,27/12/2024 10:55:44,119.25,27/12/2024 10:58:19,119.25,27/12/2024 10:58:19,120.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,EEX French Power Peak Year Future,2027-01-01,2027-01-01,2027-12-31,3132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,EEX French Power Peak Year Future,2028-01-01,2028-01-03,2028-12-29,3120.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,EEX French Power Peak Year Future,2029-01-01,2029-01-01,2029-12-31,3132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,EEX French Power Peak Year Future,2030-01-01,2030-01-01,2030-12-31,3132.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
